# Phase de Nettoyage des données

Les différentes données récupérées ont tout d'abord besoin d'être nettoyées pour les besoins du projet. Si les données fournies par l'EUDA sont déjà très propres et ne nécéssiteront qu'un léger travail de tri, voire éventuellement quelques merge utiles sur certaines tables, ce n'est pas le cas des données scrapées à partir de pages Wikipedia qui vont nous demander un travail de nettoyage plus approfondi.

## Nettoyage des données scrapées depuis Wikipedia

On commence par générer notre dataframe à partir du scraping des données de [l'article Wikipedia sur la légalisation du Cannabis par Pays](https://fr.wikipedia.org/wiki/L%C3%A9gislation_sur_le_cannabis). Le code détaillé et commenté se trouve dans le fichier `scraping.py`, on se contente ici d'appeler nos fonctions.


In [1]:
from Data.scraping import scraping as s

url = "https://fr.wikipedia.org/wiki/L%C3%A9gislation_sur_le_cannabis"

table = s.scrape_first_table(url)
data, header = s.table_to_list(table)
legal_pays = s.make_pandas(data, header)

legal_pays.head()

{'Albanie': ['Illégale', 'Illégale', 'Illégal', 'Illégale', '[13]']}
{'Albanie': ['Illégale', 'Illégale', 'Illégal', 'Illégale', '[13]'], 'Algérie': ['Illégale', 'Illégale', 'Illégal', 'Illégale', 'Des autorisations pour usages médicaux et de recherche scientifique sont possibles.Article détaillé\xa0: Cannabis en Algérie.']}
{'Albanie': ['Illégale', 'Illégale', 'Illégal', 'Illégale', '[13]'], 'Algérie': ['Illégale', 'Illégale', 'Illégal', 'Illégale', 'Des autorisations pour usages médicaux et de recherche scientifique sont possibles.Article détaillé\xa0: Cannabis en Algérie.'], 'Allemagne': ["Légale jusqu'à 25 grammes", 'Légale pour les associations agréés', 'Légale', "Légale jusqu'à trois plants par personnes", "Article détaillé\xa0: Cannabis en Allemagne. Il existe un mythe selon lequel la possession de petites quantités de haschich ou de marijuana a été légalisée par le Tribunal constitutionnel fédéral en 1994. Ce n'est pas vrai. Le tribunal a seulement fait remarquer que d'éventuel

,Possession,Vente,Transport,Culture,Notes
Albanie,Illégale,Illégale,Illégal,Illégale,[13]
Algérie,Illégale,Illégale,Illégal,Illégale,Des autorisations pour usages médicaux et de r...
Allemagne,Légale jusqu'à 25 grammes,Légale pour les associations agréés,Légale,Légale jusqu'à trois plants par personnes,Article détaillé : Cannabis en Allemagne. Il e...
Arabie saoudite,Illégale,,,,Article détaillé : Cannabis en Arabie saoudite.
Argentine,Illégale,Illégale,Illégal,Illégale sauf à des fins médicinales depuis le...,Article détaillé : Cannabis en Argentine.


On remarque alors que notre Dataframe contient plusieurs éléments qui nécéssitent d'être nettoyés, modifiés, voire supprimés. Dans l'ordre on peut citer:

1. **Supprimer la dernier colonne concernant les "Notes"**, qui peuvent nous être utile à des fins d'interprétation, ou encore pour aider à la classification ou à l'interprétation plus tard, mais qui en l'état seront inutiles à la statistique descriptive ou à la construction de modèles plus tard.

2. **Eliminer de notre dataframe les pays non-européens**, puisque les données de l'EUDA ne s'intéresse qu'au cas européeen, et que c'est là le coeur de notre sujet.

3. Enfin, il nous faudra **regrouper les différentes états des variables sous un format clair** En l'état, on dispose pour chaque pays d'une description parfois unique et idividuelle de la législation dans le pays donné. L'idéal serait de ne garder qu'une variable binaire: Légal/Illégal. Cependant la réalité est plus complexe dans certains pays, et il nous faudra alors faire des choix, et les justifier, pour essayer de ne garder qu'un nombre limité d'état pour chaque variable.

## Suppression de la colonne des Notes

Cela n'est pas bien compliqué, on utilise simplement les outils de la librairie Pandas:

In [2]:
import pandas as pd

legal_pays_clean = legal_pays.drop(columns = "Notes")

legal_pays_clean.head()

,Possession,Vente,Transport,Culture
Albanie,Illégale,Illégale,Illégal,Illégale
Algérie,Illégale,Illégale,Illégal,Illégale
Allemagne,Légale jusqu'à 25 grammes,Légale pour les associations agréés,Légale,Légale jusqu'à trois plants par personnes
Arabie saoudite,Illégale,,,
Argentine,Illégale,Illégale,Illégal,Illégale sauf à des fins médicinales depuis le...


## Tri pour ne garder que les pays Européens

Cette opération n'est elle aussi pas très compliquée, on va se servir d'une table de correspondance scrapée sur le [site national de Statistiques Candiennes](https://www23.statcan.gc.ca/imdb/p3VD.pl?Function=getVD&TVD=1312110) (ce afin d'avoir le nom des pays en anglais), qui nous sera d'ailleurs aussi utile plus tard afin de faire correspondre chaque pays de l'Union européenne avec son code ISO 3316.

In [3]:
from Data.scraping import scraping as s

url = "https://www23.statcan.gc.ca/imdb/p3VD.pl?Function=getVD&TVD=1312110"

table = s.scrape_first_table(url)
data, header = s.table_to_list(table)
table_corres = s.make_pandas(data)

table_corres.head()

{'Belgium': ['BE', 'BEL']}
{'Belgium': ['BE', 'BEL'], 'Bulgaria': ['BG', 'BGR']}
{'Belgium': ['BE', 'BEL'], 'Bulgaria': ['BG', 'BGR'], 'Croatia': ['HR', 'HRV']}
{'Belgium': ['BE', 'BEL'], 'Bulgaria': ['BG', 'BGR'], 'Croatia': ['HR', 'HRV'], 'Cyprus': ['CY', 'CYP']}
{'Belgium': ['BE', 'BEL'], 'Bulgaria': ['BG', 'BGR'], 'Croatia': ['HR', 'HRV'], 'Cyprus': ['CY', 'CYP'], 'Czechia': ['CZ', 'CZE']}
{'Belgium': ['BE', 'BEL'], 'Bulgaria': ['BG', 'BGR'], 'Croatia': ['HR', 'HRV'], 'Cyprus': ['CY', 'CYP'], 'Czechia': ['CZ', 'CZE'], 'Denmark': ['DK', 'DNK']}
{'Belgium': ['BE', 'BEL'], 'Bulgaria': ['BG', 'BGR'], 'Croatia': ['HR', 'HRV'], 'Cyprus': ['CY', 'CYP'], 'Czechia': ['CZ', 'CZE'], 'Denmark': ['DK', 'DNK'], 'Estonia': ['EE', 'EST']}
{'Belgium': ['BE', 'BEL'], 'Bulgaria': ['BG', 'BGR'], 'Croatia': ['HR', 'HRV'], 'Cyprus': ['CY', 'CYP'], 'Czechia': ['CZ', 'CZE'], 'Denmark': ['DK', 'DNK'], 'Estonia': ['EE', 'EST'], 'Finland': ['FI', 'FIN']}
{'Belgium': ['BE', 'BEL'], 'Bulgaria': ['BG', 'BGR'], 

,0,1
Belgium,BE,BEL
Bulgaria,BG,BGR
Croatia,HR,HRV
Cyprus,CY,CYP
Czechia,CZ,CZE


On doit néanmoins ajouter à la main certains pays non-membres de l'Union Européenne, mais présent dans les données que nous avons à disposition (souvent des pays candidats comme la Turquie)

In [6]:
import pandas as pd

table_corres.loc["Austria"] = ["AT","AUT"]
table_corres.loc["Türkiye"] = ["TR","TUR"]
table_corres.loc["Norway"] = ["NO","NOR"]

table_corres.tail()

,0,1
Spain,ES,ESP
Sweden,SE,SWE
Austria,AT,AUT
Türkiye,TR,TUR
Norway,NO,NOR


# Ajout de nouvelles tables

**Tables de données sur les "entrants" pour des soins liés à la consommation de cannabis**

Définition :
**Entrants** : Personnes ayant entamé au cours de l'année un parcours de soin lié à un consommation de drogue (de cannabis ici).

On commence donc par "nettoyer" les tables principales en ne gardant à chaque fois que les colonnes qui nous intéressent :
- Pays
- Année
- Total d'entrants
- Nombre d'entrants liés au cannabis

In [32]:
import pandas as pd
import openpyxl

### On commence par les donnees generales
total = "/home/onyxia/work/Projet_Python_2A/Data/EUDA/Cannabis_detaille/drogues_entrants.xlsx"
df1 = pd.read_excel(total, header=3)

# On ne garde que les colonnes qui nous intéressent
colonnes = ['Country', 'Year of Treatment', 'All substances', 'Cannabis']
df1 = df1[colonnes]


### On procede de la meme maniere pour les donnees concernant les personnes deja traitees auparavant
deja = "/home/onyxia/work/Projet_Python_2A/Data/EUDA/Cannabis_detaille/deja_traite_entrants.xlsx"
df2 = pd.read_excel(deja, header=3)

# On ne garde que les colonnes qui nous intéressent
colonnes = ['Country', 'Year of Treatment', 'All substances', 'Cannabis']
df2 = df2[colonnes]


### On procede de la meme maniere pour les donnees concernant les personnes jamais traitees auparavant
jamais = "/home/onyxia/work/Projet_Python_2A/Data/EUDA/Cannabis_detaille/jamais_traite_entrants.xlsx"
df3 = pd.read_excel(jamais, header=3)

# On ne garde que les colonnes qui nous intéressent
colonnes = ['Country', 'Year of Treatment', 'All substances', 'Cannabis']
df3 = df3[colonnes]


/opt/conda/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/opt/conda/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/opt/conda/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


**On merge ces trois tables qui vont constituer la base de notre table sur les "entrants"**
Une fois les infos globales sur les "entrants" dans chaque pays d'Europe groupées pour former une table, on rajoutera l'ensemble des données que l'on possède sur le sujet.
On commencera par effectuer des **statistiques descriptives** puis ces données serviront à entraîner notre modèle de **machin learning**.

In [33]:
### On effectue le merge (left join) afin de grouper les données par pays

## On commence par un merge "intermédiaire"
# On commence par supprimer la colonne "year of treatment" pour df 2 car cette colonne apparaît dans les deux tables
df2 = df2.drop(columns=["Year of Treatment"])
intermediaire = pd.merge(df1, df2, how="left", on="Country")
# Il faut renommer certaines colonnes (noms en doublon)
intermediaire = intermediaire.rename(columns={
    "Country": "Country",
    "Year of Treatment": "Year of Treatment",
    "All substances_x": "All substances (global)",
    "Cannabis_x": "Cannabis (global)",
    "All substances_y": "All substances (deja traite.es)",
    "Cannabis_y": "Cannabis (deja traite.es)"
})

## On reproduit les mêmes étapes mais en "mergeant" intermediaire et df3 pour finaliser cette table de BaseException
# On commence par supprimer la colonne "year of treatment" pour df 2 car cette colonne apparaît dans les deux tables
df3 = df3.drop(columns=["Year of Treatment"])
table_base = pd.merge(intermediaire, df3, how="left", on="Country")
# Il faut renommer certaines colonnes (noms en doublon)
table_base = table_base.rename(columns={
    "Country": "Country",
    "Year of Treatment": "Year of Treatment",
    "All substances (global)": "All substances (global)",
    "Cannabis (global)": "Cannabis (global)",
    "All substances (deja traite.es)": "All substances (deja traite.es)",
    "Cannabis (deja traite.es)": "Cannabis (deja traite.es)",
    "All substances": "All substances (jamais traite.es)",
    "Cannabis": "Cannabis (jamais traite.es)",
})

